In [1]:
# Dependencies and libraries
import requests
import pandas as pd

from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Juanjo\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


# Scrapping redplanet science

In [3]:
url = 'https://redplanetscience.com'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())


<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [5]:
# Looking at the browser inspector this line contains the title:
# <div class="content_title">NASA's Mars 2020 Heads Into the Test Chamber</div>
# And this line contains the teaser of the article
# <div class="article_teaser_body">In this time-...</div>

# Saving the results
title = soup.find('div', class_='content_title').text
teaser = soup.find('div', class_='article_teaser_body').text


# Scrapping spaceimages-mars.com

In [6]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)
browser.links.find_by_partial_text('FULL').click()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<html class="fancybox-margin fancybox-lock">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
  <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <title>
   Space Image
  </title>
  <style type="text/css">
   .fancybox-margin{margin-right:17px;}
  </style>
 </head>
 <body>
  <div class="header">
   <nav class="navbar navbar-expand-lg">
    <a class="navbar-brand" href="#">
     <img id="logo" src="image/nasa.png"/>
     <span class="logo">
      Jet Propulsion Laboratory
     </span>
     <span class="logo1">
      California Institute of Technology
     </span>
    </a>
    <button aria-controls="navbarNav" aria-expand

In [7]:
# Full size image is at
# <img class="fancybox-image" src="image/featured/mars2.jpg" alt="">
image = soup.find('img', class_='fancybox-image')

# Building the full link
image_link = 'https://spaceimages-mars.com/' + (image['src'])


In [8]:
# Scrapping Mars facts
url = 'https://galaxyfacts-mars.com/'  

tables = pd.read_html(url, skiprows=1)
mars_facts = tables[0]

# Dropping columns, renaming and indexing
mars_facts = mars_facts.drop(columns=[2])
mars_facts = mars_facts.rename(columns={0: 'Fact', 1: 'Information'})

#Saving to html table
facts_table = mars_facts.to_html(index = False)

#Replacing the \n
facts_table = facts_table.replace('\n', '')
facts_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Fact</th>      <th>Information</th>    </tr>  </thead>  <tbody>    <tr>      <td>Diameter:</td>      <td>6,779 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>    </tr>    <tr>      <td>Moons:</td>      <td>2</td>    </tr>    <tr>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>    </tr>    <tr>      <td>Length of Year:</td>      <td>687 Earth days</td>    </tr>    <tr>      <td>Temperature:</td>      <td>-87 to -5 °C</td>    </tr>  </tbody></table>'

# Scrapping mars hemispheres

In [9]:
url = 'https://marshemispheres.com/'
browser.visit(url)

# Hemisphere list
hemi_list = ['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']

# For loop to iterate through all the hemisphere's pages

# List to store results
hemi_img = []


for _ in hemi_list:

    # Clicking the corresponding link
    browser.links.find_by_partial_text(_).click()
    html = browser.html
    
    # Scrapping the hemisphere page
    soup = BeautifulSoup(html, 'html.parser')
    img_url = soup.find('div', class_='downloads')
    links_list = []
    
    # Retrieving image link
    for link in img_url.ul.find_all('a'):
        links_list.append(link.get('href'))
    img_link = 'https://marshemispheres.com/' + links_list[1]
    
    # Retrieving and formatting title
    title = soup.find('h2', class_='title').get_text()
    title = title.replace(' Enhanced', '')

    # Saving the results as a dictionary and appending to list
    res_dict = {'title': title, 'img_url': img_link}
    hemi_img.append(res_dict)
    
    # Resetting for next iteration
    browser.links.find_by_partial_text('Back').click()    
    

In [10]:
browser.quit()

# Creating the final dictionary to return

In [11]:
scrapped_data = {
    'latest_news': title,
    'teaser': teaser,
    'featured_image': image_link,
    'mars_facts': facts_table,
    'hemisphere_images': hemi_img
}

